# CADP tools queries 

## 2. SVL queries

In [1]:
# Choose target graph 
file_name = "sample.bcg"
file_path = f"../graph/bcg/{file_name}"

In [16]:
before_promo_period = "2018-09-01"
promo_period ="2018-12-01"
after_promo_period = ":2019-02-01"

### 2.1 Adoption query

In [66]:
query_name = "product_adoption.svl"
query_path = f"../queries/svl/{query_name}"
file_path = f'../graph/bcg/9M-5-[1-2001]-[a]-lcm'

In [67]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl product_adoption.svl $1 ":3461025:7461024:" ":2018-09-01" ":2018-12-01" ":2019-02-01"


property PRODUCT_ADOPTION
 | Product adoption

TRUE

[Customers] [Promotional products] Period

[211249] [7461024] :2018-12-01
[264552] [3461025] :2018-12-01
[277744] [3461025] :2018-12-01
[322520, 526900] [3461025] :2018-12-01
[322520, 575459] [3461025] :2018-12-01
[336921, 575459] [3461025] :2018-12-01
[400827, 526900, 575459] [3461025] :2018-12-01
[400827, 575459, 1044697] [3461025] :2018-12-01
[400827, 575459] [3461025] :2018-12-01
[526900, 575459] [3461025] :2018-12-01
[526900] [3461025] :2018-12-01
[575459, 1044697] [3461025] :2018-12-01
[575459, 819320] [3461025] :2018-12-01
[575459] [3461025] :2018-12-01
[575459] [7461024] :2018-12-01
[819320] [3461025] :2018-12-01



###  2.2 Loyalty query 

In [88]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl product_loyalty.svl $1 ":3461025:7461024:" ":2018-12-01" ":2019-02-01"


property product_loyalty

TRUE

[Customers] [Promotional products] Period

[1044697] [3461025] :2018-12-01
[211249] [7461024] :2018-12-01
[575459, 1044697, 1163180] [3461025] :2018-12-01
[575459] [3461025] :2018-12-01



### 2.3 Pomotion only query

In [108]:
%%bash -s $file_path
export PATH=$PATH:$CADP/bin.x64:$CADP/com
$CADP/com/svl promotion_only.svl $1 ":2018-09-01" ":2018-12-01" ":2019-02-01"


property promotion_only

TRUE

[Customers] [Promotional products] Period




In [109]:
!gedit promotion_only.svl
